In [3]:
from util.VisualizeDataset import VisualizeDataset
from Chapter7.PrepareDatasetForLearning import PrepareDatasetForLearning
from Chapter7.Evaluation import RegressionEvaluation, ClassificationEvaluation
from Chapter8.LearningAlgorithmsTemporal import TemporalClassificationAlgorithms
from Chapter8.LearningAlgorithmsTemporal import TemporalRegressionAlgorithms
from Chapter7.FeatureSelection import FeatureSelectionClassification
from statsmodels.tsa.stattools import adfuller
from pandas.plotting import autocorrelation_plot
# from exercises_ch7_classification_individual import used_features

import sys
import copy
import pandas as pd
from util import util
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
from sklearn.model_selection import train_test_split

In [41]:
from pathlib import Path
import pandas as pd
from util.VisualizeDataset import VisualizeDataset

DATA_PATH = Path('./intermediate_datafiles/')
DATASET_FNAME1 = 'chapter5_resultIvo.csv'
DATASET_FNAME2 = 'chapter5_resultJoost.csv'
DATASET_FNAME3 = 'chapter5_resultFlo.csv'
# RESULT_FNAME =  'chapter3_result_outliers'+participant+'.csv'

# Next, import the data from the specified location and parse the date index.
try:
    dataset1 = pd.read_csv(DATA_PATH / DATASET_FNAME1, index_col=0)
    dataset2 = pd.read_csv(DATA_PATH / DATASET_FNAME2, index_col=0)
    dataset3 = pd.read_csv(DATA_PATH / DATASET_FNAME3, index_col=0)
except IOError as e:
    print('File not found, try to run previous crowdsignals scripts first!')
    raise e

common_columns = set(dataset1.columns) & set(dataset2.columns) & set(dataset3.columns)
dataset1 = dataset1[common_columns]
dataset2 = dataset2[common_columns]
dataset3 = dataset3[common_columns]

dataset = pd.concat([dataset1, dataset2, dataset3], ignore_index=False)
dataset.index = pd.to_datetime(dataset.index)

# We'll create an instance of our visualization class to plot the results.
DataViz = VisualizeDataset()

# Of course we repeat some stuff from Chapter 3, namely to load the dataset

# Read the result from the previous chapter, and make sure the index is of the type datetime.


# Let us consider our second task, namely the prediction of the heart rate. We consider this as a temporal task.

prepare = PrepareDatasetForLearning()

train_X, test_X, train_y, test_y = prepare.split_single_dataset_classification(dataset, ['label'], 'like', 0.7, filter=True, temporal=True)
train_y = pd.get_dummies(train_y)
test_y = pd.get_dummies(test_y)

In [49]:
print(np.sum(train_y, axis=1))
print(np.sum(test_y, axis=1))



2016-02-08 18:28:30.860764416    1
2016-02-08 18:28:31.260764416    1
2016-02-08 18:28:31.660764416    1
2016-02-08 18:28:32.060764416    1
2016-02-08 18:28:32.460764416    1
                                ..
2016-02-08 18:28:56.058355968    1
2016-02-08 18:28:56.458355968    1
2016-02-08 18:28:56.858355968    1
2016-02-08 18:28:57.258355968    1
2016-02-08 18:28:57.658355968    1
Length: 965, dtype: int64
2016-02-08 18:28:58.058355968    1
2016-02-08 18:28:58.458355968    1
2016-02-08 18:28:58.858355968    1
2016-02-08 18:28:59.258355968    1
2016-02-08 18:28:59.658355968    1
                                ..
2016-02-08 18:31:42.058355968    1
2016-02-08 18:31:42.458355968    1
2016-02-08 18:31:42.858355968    1
2016-02-08 18:31:43.258355968    1
2016-02-08 18:31:43.658355968    1
Length: 415, dtype: int64


In [45]:
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN
import numpy as np

num_classes = 7
batch_size = 32
feature_dim = 427

# Reshape your input data to have the appropriate shape
train_X = np.random.random([965, feature_dim]).astype(np.float32)
train_X = np.reshape(train_X, (train_X.shape[0], 1, train_X.shape[1]))  # Reshape to (samples, timesteps, features)

test_X = np.random.random([415, feature_dim]).astype(np.float32)
test_X = np.reshape(test_X, (test_X.shape[0], 1, test_X.shape[1]))  # Reshape to (samples, timesteps, features)

mmodel = Sequential()

num_categories = 7
# Define the model
model = Sequential()
model.add(SimpleRNN(num_classes, activation='relu', input_shape=(None, feature_dim)))  # RNN layer
model.add(Dense(num_categories, activation='softmax'))  # Output layer

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(train_X, train_y, epochs=10, batch_size=batch_size)

# Evaluate the model
loss, accuracy = model.evaluate(test_X, test_y)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

# Make predictions
predictions = model.predict(test_X)


Epoch 1/10
31/31 [==============================] - 1s 1ms/step - loss: 1.9496 - accuracy: 0.1668
Epoch 2/10
31/31 [==============================] - 0s 1ms/step - loss: 1.9280 - accuracy: 0.1979
Epoch 3/10
31/31 [==============================] - 0s 1ms/step - loss: 1.9208 - accuracy: 0.2104
Epoch 4/10
31/31 [==============================] - 0s 1ms/step - loss: 1.9132 - accuracy: 0.2052
Epoch 5/10
31/31 [==============================] - 0s 1ms/step - loss: 1.9114 - accuracy: 0.2176
Epoch 6/10
31/31 [==============================] - 0s 1ms/step - loss: 1.9072 - accuracy: 0.2269
Epoch 7/10
31/31 [==============================] - 0s 1ms/step - loss: 1.9003 - accuracy: 0.2290
Epoch 8/10
31/31 [==============================] - 0s 1ms/step - loss: 1.9063 - accuracy: 0.2073
Epoch 9/10
31/31 [==============================] - 0s 1ms/step - loss: 1.8926 - accuracy: 0.2218
Epoch 10/10
31/31 [==============================] - 0s 1ms/step - loss: 1.8905 - accuracy: 0.2332


ValueError: in user code:

    File "/Users/florencecornelissen/opt/anaconda3/envs/ML4QS2023/lib/python3.8/site-packages/keras/engine/training.py", line 1852, in test_function  *
        return step_function(self, iterator)
    File "/Users/florencecornelissen/opt/anaconda3/envs/ML4QS2023/lib/python3.8/site-packages/keras/engine/training.py", line 1836, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/florencecornelissen/opt/anaconda3/envs/ML4QS2023/lib/python3.8/site-packages/keras/engine/training.py", line 1824, in run_step  **
        outputs = model.test_step(data)
    File "/Users/florencecornelissen/opt/anaconda3/envs/ML4QS2023/lib/python3.8/site-packages/keras/engine/training.py", line 1790, in test_step
        self.compute_loss(x, y, y_pred, sample_weight)
    File "/Users/florencecornelissen/opt/anaconda3/envs/ML4QS2023/lib/python3.8/site-packages/keras/engine/training.py", line 1109, in compute_loss
        return 

In [ ]:
# Step 1: Import necessary libraries
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense

# Step 2: Prepare your data
# Assume you have your input data X and corresponding labels y ready

# Step 3: Preprocess your data
# Normalize your input data, reshape it, and one-hot encode your labels if necessary

# Step 4: Split your data into training and testing sets
# Assuming you have X_train, y_train, X_test, y_test

# Step 5: Build the LSTM model
model = Sequential()
model.add(LSTM(64, input_shape=(time_steps, num_features)))
model.add(Dense(num_classes, activation='softmax'))

# Step 6: Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Step 7: Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Step 8: Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test, batch_size=32)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)
